Università degli Studi di Palermo

Corso di Laurea Magistrale in **Ingegneria Informatica**

**Assignment 1** di **Visione Artificiale** - **Strumento per la navigazione di immagini a 360°**

Sviluppato da **Castelli Giovanni**, **Costa Gabriele Nicolò**, **Sollazzo Ivan**.

In [12]:
# Importazione delle librerie necessarie
import numpy as np
import cv2

# Funzioni immagini

### Funzione per la riproduzione di un video

In [13]:
def open_video(path):

    # Importazione del video
    video = cv2.VideoCapture(path)

    # Riproduci il video fintantoché aperto
    while video.isOpened():

        # Leggi frame per frame
        ret, frame = video.read()

        # Verifica se c'è il frame
        if ret:
            # Visualizza il frame
            cv2.imshow("Frame", frame)
        else:
            print("Errore nella lettura del frame.")

        # Verifica se il tasto premuto è q
        if cv2.waitKey(25) == ord('q'):
            print("Uscita in corso dal video...")

            # Rilascia il video e distruggi tutte le finestre
            video.release()
            cv2.destroyAllWindows()

### Funzione per la cattura di un frame

In [14]:
def capture_frame(path, frameNumber = 1):
    # Importazione del video
    video = cv2.VideoCapture(path)

    # Contatore frame
    frameCount = frameNumber

    # Verifica se il video è aperto correttamente
    if video.isOpened():
        while video.isOpened():

            # Posizionati sul frame frameCount - 1
            video.set(cv2.CAP_PROP_POS_FRAMES, frameCount - 1)

            # Leggi il frame
            ret, frame = video.read()

            if ret:
                video.release()
                return frame
            else:
                print("Errore durante la lettura del frame.")
                frameCount += 1

    else:
        print("Errore durante l'apertura del video.")

### Funzione per l'inverse mapping

In [15]:
def inverse_map(input_image, w_output, h_output, latitude, longitude, fov):

    # Calcolo dimensioni immagine equirettangolare
    h_input, w_input = input_image.shape[0], input_image.shape[1]

    # Conversione da gradi a radianti
    latitude_rad = np.radians(latitude)
    longitude_rad = np.radians(longitude)
    fov_rad = np.radians(fov)

    # Creazione di un piano tangente scalato
    u = np.linspace(-1, 1, w_output) * np.tan(fov_rad / 2)
    v = np.linspace(-1, 1, h_output) * np.tan(fov_rad / 2)
    U, V = np.meshgrid(u, v)

    # Costruzione dei vettori nel piano tangente, assumendo x = 1
    X = np.ones_like(U) * zoom
    Y = U.copy()
    Z = -V.copy()

    # Definizione delle matrici di rotazione
    Ry = np.array([[np.cos(latitude_rad), 0, np.sin(latitude_rad)], [0, 1, 0], [-np.sin(latitude_rad), 0, np.cos(latitude_rad)]])
    Rz = np.array([[np.cos(longitude_rad), -np.sin(longitude_rad), 0], [np.sin(longitude_rad), np.cos(longitude_rad), 0], [0, 0, 1]])

    # Costruzione dei vettori intersecanti il piano
    V_initial = np.stack((X, Y, Z), axis=-1)

    # Rotazione del piano
    V_rotated = V_initial @ Ry @ Rz

    # Normalizzazione
    V_normalized = V_rotated / np.linalg.norm(V_rotated, axis=-1, keepdims=True)

    # Calcolo delle coordinate sferiche
    theta = np.arccos(V_normalized[..., 2])
    phi = np.arctan2(V_normalized[..., 1 ], V_normalized[...,0])

    # Mappatura alle coordinate dell'immagine equirettangolare
    map_x = (((phi / np.pi) + 1) * (w_input / 2)).astype(np.float32)
    map_y  = ((theta / (np.pi)) * h_input).astype(np.float32)

    # Creazione di un'immagine planare
    output_image = np.zeros((h_output, w_output, 3), dtype=np.uint8)

    # Utilizzo di remap per effettuare l'interpolazione
    output_image = cv2.remap(input_image, map_x, map_y, interpolation=cv2.INTER_LINEAR)

    return output_image

### Funzione per gestire l'input da tastiera

In [16]:
# Parametri da gestire
lat = 0
long = 0
FOV = 60
zoom = 1
w = 500 #lunghezza dell'immagine
h = 500 #altezza dell'immagine

# Funzione per la gestione dell'input da tastiera
def tastiera():
    global lat, long, FOV, zoom, w, h, paused
    # Aspetta l'input da tastiera
    key = cv2.waitKey(1) & 0xFF
    #print(key) utile per capire il codice del tasto che clicco
    # Se la freccia su è premuta
    if key == ord('w'):  
        lat += 1
    # Se la freccia giù è premuta
    elif key == ord('s'):  
        lat -= 1
    # Se la freccia destra è premuta
    elif key == ord('a'):  
        long += 1
    # Se la freccia sinistra è premuta
    elif key == ord('d'): 
        long -= 1
    # Se il tasto c è premuto zooma
    elif key == ord('c'):  
        zoom +=0.1
        if zoom > 10:
            zoom = 10  
    # Se il tasto v è premuto dezooma
    elif key == ord('v'): 
        zoom -= 0.1
        if zoom < 1:
            zoom = 1
    # Se il tasto x è premuto aumenta il FOV
    elif key == ord('x'):
        FOV += 1
    # Se il tasto z è premuto diminuisci il FOV
    elif key == ord('z'):
        FOV -= 1
    # Se il tasto 'q' è premuto, esce dal ciclo
    elif key == ord('q'):
        return False
    elif key == ord('t'):
        if not paused:
            paused = True
        else:
            paused = False
    return True

### Funzione per mostrare i frame

In [17]:
# Funzione per mostrare i frame
def show_frame(frame, title="Frame"):
    # Mostra il frame
    cv2.imshow(title, frame)
    if not tastiera(): #se tastiera restituisce False (ovvero premo q)
        cv2.destroyAllWindows() #distruggo l'immagine
        return False
    return True

### Funzione per mostrare l'immagine planare

In [18]:
def show_planar(frame):
    bool = True
    while bool:
        plane_image = inverse_map(frame, w, h, lat, long, FOV) #genero la nuova immagine
        if not show_frame(plane_image):
            bool = False     

# Demo immagine

In [19]:
# Cattura del frame
frame = capture_frame("video_1.MP4")

# Mostra il frame
show_planar(frame)

# Funzioni video

### Funzione per la cattura dei frame di un video in un array

In [20]:
def capture_video_frames(path):
    video = cv2.VideoCapture(path)
    nFrames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    frames = []

    for i in range(1, nFrames):
        frame = capture_frame(path, i)
        print(f"Saving frame {i} of {int(nFrames)}...")
        frames.append(frame)
    
    return frames

### Funzione per la riproduzione del video

In [21]:
def play_video(video_array):

    # Metodo di processamento del frame
    def process_frame(frame):
        plane_image = inverse_map(frame, w, h, lat, long, FOV)
        return plane_image
    
    # Loop di esecuzione
    i = 0
    running = True

    while running:
        if i < len(video_array):
            frame = video_array[i]
            plane_image_final = process_frame(frame)

            if not show_frame(plane_image_final):
                break
            if paused:
                while True:
                    plane_image_final = process_frame(video_array[i])
                    if not show_frame(plane_image_final):
                        running = False
                        break
                    if not paused:
                        break
                if not running:
                    break
            i += 1
        else:
            cv2.destroyAllWindows()
            running = False

# Demo video

In [22]:
paused = False
video_array = capture_video_frames("video_1.MP4")
play_video(video_array)

Saving frame 1 of 1140...
Saving frame 2 of 1140...
Saving frame 3 of 1140...
Saving frame 4 of 1140...
Saving frame 5 of 1140...
Saving frame 6 of 1140...
Saving frame 7 of 1140...
Saving frame 8 of 1140...
Saving frame 9 of 1140...
Saving frame 10 of 1140...
Saving frame 11 of 1140...
Saving frame 12 of 1140...
Saving frame 13 of 1140...
Saving frame 14 of 1140...
Saving frame 15 of 1140...
Saving frame 16 of 1140...
Saving frame 17 of 1140...
Saving frame 18 of 1140...
Saving frame 19 of 1140...
Saving frame 20 of 1140...
Saving frame 21 of 1140...
Saving frame 22 of 1140...
Saving frame 23 of 1140...
Saving frame 24 of 1140...
Saving frame 25 of 1140...
Saving frame 26 of 1140...
Saving frame 27 of 1140...
Saving frame 28 of 1140...
Saving frame 29 of 1140...
Saving frame 30 of 1140...
Saving frame 31 of 1140...
Saving frame 32 of 1140...
Saving frame 33 of 1140...
Saving frame 34 of 1140...
Saving frame 35 of 1140...
Saving frame 36 of 1140...
Saving frame 37 of 1140...
Saving fra